In [11]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from typing import Dict
from typing import List

In [16]:
class Point:
    id = 1
    def __init__(self,a,b, shake=1e-5):
        self.id = Point.id
        Point.id+=1
        shakes = np.random.uniform(-shake, shake, 2)
        self.x = a+shakes[0]
        self.y = b+shakes[1]

    def __add__(self, other):
        if isinstance(other, Point):
            return Point(self.x+other.x, self.y+other.y)
        else:
            raise TypeError("Unsupported operand type")
        
    def __sub__(self, other):
        if isinstance(other, Point):
            return Point(self.x-other.x, self.y-other.y)
        else:
            raise TypeError("Unsupported operand type")
        
    def __str__(self):
        return str(self.id)+": "+str(self.x)+"   "+str(self.y)
    
    def __lt__(self,other):
        if isinstance(other, Point):
            vec_prod = (self.x*other.y)-(self.y*other.x)
            if vec_prod>0:
                return True
            elif vec_prod<0:
                return False
            else:
                raise Exception("Segments are parallel")
        else:
            raise TypeError("Unsupported operand type")
        
    def Reset():
        Point.id = 1

In [4]:
class Segment:
    id = 0

    def __init__(self,a: Point, b: Point):
        if b<a:
            temp = a
            a = b
            b = temp
        self.bgn = a
        self.end = b
        self.inclination = (self.end.y-self.bgn.y)/(self.end.x-self.bgn.x)
        self.intercept = self.end.y - (self.inclination * self.end.x)
        self.id = 0 #Usado apenas para indicar ordem durante o algoritmo de varredura linear
    
    def Y_At_X(self,x):
        return self.inclination*x+self.intercept


    def Invert(self):
        temp = self.bgn
        self.bgn = self.end
        self.end = temp

    def __str__(self):
        return "Begin: "+str(self.bgn.x)+"   "+str(self.bgn.y)+"\nEnd: "+str(self.end.x)+"   "+str(self.end.y)+"\nInclination: "+str(self.inclination)
    
    def __lt__(self,other):
        if isinstance(other, Segment):
            return self.bgn.x < other.bgn.x
        else:
            raise TypeError("Unsupported operand type")

## Primitivas

In [5]:
def Clockwise(a: Segment, b: Segment): #returns true if a is clockwise to b
    frst_seg = a.end - a.bgn
    scnd_seg = b.end - b.bgn
    vec_prod = (frst_seg.x*scnd_seg.y)-(frst_seg.y*scnd_seg.x)
    if vec_prod>0:
        return True
    elif vec_prod<0:
        return False
    else:
        raise Exception("Segments are parallel")

In [6]:
def SegmentsIntercept(a: Segment,b: Segment):
    check_one = Clockwise(a,Segment(a.bgn,b.bgn))
    check_two = Clockwise(a,Segment(a.bgn,b.end))
    if check_one==check_two:
        return False
    check_one = Clockwise(b,Segment(b.bgn,a.bgn))
    check_two = Clockwise(b,Segment(b.bgn,a.end))
    if check_one==check_two:
        return False
    return True

## Data reading and pre-processing

In [12]:
def SetTwoDimensions(df):
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(df)
    df_pca = pd.DataFrame(data = pca_result, columns = ['x', 'y'])
    return df_pca

In [13]:
def PreProcessData(path, tgtCol):
    df = pd.read_csv(path)
    tgtData = df.iloc[:, tgtCol]  # Save the nth column
    df = df.drop(df.columns[tgtCol], axis=1)
    df = SetTwoDimensions(df)
    df['target_data'] = tgtData
    return df

## Sweep Line

In [7]:
#Para a varredura funcionar, eu estou supondo que o vetor de valores de x à varrer está ordenado e que o vetor de segmentos foi corretamente pré-computado, 
#isto é, o vetor de segmentos deve ser ordenado pela função de ordem já definida, não existem 2 pontos extremos com o mesmo valor de x e o segmento que ocupa a 
#i-ésima posição do vetor de segmentos deve ter id=i

#Devido aos cuidados nescessários com isso, também já comecei a implementar as funções que leem os dados e pré-computam tudo para o SweepLine rodar fino
#PS: QUE ALGORITMO FEIO DO CARALHO

def SweepLine(segment_vector: List[Segment], x_vector):
    j = 0
    comp = dict()
    ans = dict()
    scope = []
    for i in range(len(x_vector)):
        seg = segment_vector[j]
        #olhar no escopo se algum segmento já alcançado começa em x==i
        if seg.bgn==i:
            scope.append(seg)
            j+=1
            SegmentEnteringComparison(seg,scope,comp)
        #olhar no escopo qual dos segmentos termina em x==i
        else:
            for k in scope:
                if k.end==i:
                    #compara tudo
                    SegmentLeftingComparison(i,seg.id,segment_vector,scope,comp,ans)
                    scope.remove(k)
    return ans

def SegmentEnteringComparison(seg: Segment, scope: List[Segment], comp: Dict[int, Dict[int, bool]]):
    above = dict()
    for i in scope:
        if seg.bgn.y<i.Y_At_X(seg.bgsn.x):
            above[i.id] = False
            comp[i.id][seg.id] = True
        else:
            above[i.id] = True
            comp[i.id][seg.id] = False
    comp[seg.id] = above

def SegmentLeftingComparison(x,
                             seg_indx: int, 
                             segment_vector: List[Segment],  
                             comp: Dict[int, Dict[int, bool]], 
                             ans: Dict[int,List[int]]):
    seg = segment_vector[seg_indx]
    key = seg.id
    for i in comp[key]:
        if comp[key][i] != (seg.end.y>segment_vector[i].Y_At_X(x)):
            ans[key].append(i)
            ans[i].append(key)

In [27]:
p1 = Point(1,2)
p2 = Point(3,1)
p3 = Point(2,2)
p4 = Point(2,1)
Point.Reset()
vec_p = np.array([p1,p2,p3,p4])
vec_s = np.sort(vec_p)
for i in vec_p:
    print(i)

1: 0.9999948015166591   1.9999901810797924
2: 2.999997913822665   1.0000081665137703
3: 1.999999953488618   1.999990186458855
4: 2.000008793410377   1.0000003650231482


In [30]:
for i in vec_s:
    print(i)

2: 2.999997913822665   1.0000081665137703
4: 2.000008793410377   1.0000003650231482
3: 1.999999953488618   1.999990186458855
1: 0.9999948015166591   1.9999901810797924
